### Blake Pritchard
### DTSC-680
### Assignment-6

## Mushroom Classification

we will attempt to build a model which can determine whether a mushroom specimen is edible, based upon the quantifiable metrics observed in our mushroom database

In [1]:
# Set Up notebook dependencies

import sys
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

np.set_printoptions(threshold=sys.maxsize)

In [2]:
# instantiate encoder objects

encoder_OneHot_impute = OneHotEncoder()
encoder_Labels_impute = LabelEncoder()

encoder_OneHot_inference = OneHotEncoder()
encoder_Labels_inference = LabelEncoder()

In [3]:
# Define Column Names 

def get_column_names():
    return ['edible',
            'cap-shape',
            'cap-surface',
            'cap-color',
            'bruises',
            'odor',
            'gill-attachment',
            'gill-spacing',
            'gill-size',
            'gill-color',
            'stalk-shape',
            'stalk-root',
            'stalk-surface-above-ring',
            'stalk-surface-below-ring',
            'stalk-color-above-ring',
            'stalk-color-below-ring',
            'veil-type',
            'veil-color',
            'ring-number',
            'ring-type',
            'spore-print-color',
            'population',
            'habitat'
           ]

---
### Read Mushroom data from File

In [4]:
a_column_names = get_column_names()
mushroom_data = pd.read_csv('agaricus-lepiota.data', header=None, names=a_column_names)

In [5]:
index_rows_complete = mushroom_data['stalk-root'] != '?'
index_rows_incomplete = mushroom_data['stalk-root'] == '?'

---
### Use KNN to Impute missing values

We will start by separating the columsninto features ans labels
fot this imputation step we want to predict the values of the 'stalk-root'.
therefore, our label will be the "stalk-root' and our features will be everything else

In [6]:
impute_training_labels = mushroom_data['stalk-root']
impute_training_features = mushroom_data.loc[:, mushroom_data.columns != 'stalk-root']

Encode Features and Labels

In [7]:
impute_labels_Encoded = encoder_Labels_impute.fit_transform(impute_training_labels)
impute_features_OneHot = encoder_OneHot_impute.fit_transform(impute_training_features)


Next build an Index of Rows which Require Imputation

In [8]:
impute_training_labels_complete = impute_labels_Encoded[index_rows_complete]
impute_training_features_complete = impute_features_OneHot[index_rows_complete]
impute_predict_features_incomplete = impute_features_OneHot[index_rows_incomplete]

Train KNN Classifier on Encoded Copy of Complete Rows

In [9]:
classifier = KNeighborsClassifier()
trained_imputation_model = classifier.fit(impute_training_features_complete, impute_training_labels_complete)

Impute Missing Values Based on Encoded Copy of Incomplete Rows

In [10]:
missing_values_encoded = trained_imputation_model.predict(impute_predict_features_incomplete)

/home/blake/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [11]:
missing_values = encoder_Labels_impute.inverse_transform(missing_values_encoded)

In [12]:
np.unique(missing_values, return_counts=True)

(array(['b', 'c', 'e'], dtype=object), array([1891,   65,  524]))

### Review Value Counts

We can review the counts of each missing value 

In [13]:
unique, counts = np.unique(missing_values, return_counts=True)
print(np.asarray((unique, counts)).T)

[['b' 1891]
 ['c' 65]
 ['e' 524]]


In [14]:
missing_values

array(['b', 'b', 'e', 'b', 'b', 'b', 'b', 'b', 'b', 'e', 'e', 'b', 'b',
       'b', 'b', 'e', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'e',
       'b', 'b', 'b', 'b', 'e', 'b', 'e', 'b', 'e', 'b', 'b', 'b', 'b',
       'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b',
       'b', 'b', 'e', 'e', 'b', 'b', 'b', 'b', 'b', 'e', 'b', 'b', 'b',
       'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'e', 'b', 'b',
       'e', 'b', 'b', 'b', 'b', 'b', 'b', 'e', 'b', 'b', 'b', 'b', 'b',
       'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b',
       'b', 'b', 'b', 'b', 'e', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b',
       'b', 'b', 'b', 'b', 'b', 'b', 'b', 'e', 'b', 'b', 'b', 'b', 'b',
       'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b',
       'b', 'e', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'e', 'b', 'b', 'b',
       'b', 'b', 'e', 'b', 'b', 'b', 'b', 'b', 'b', 'e', 'e', 'b', 'e',
       'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'e', 'e', 'e

In [15]:
data_rows_complete = mushroom_data[index_rows_complete].copy()
data_rows_incomplete = mushroom_data[index_rows_incomplete].copy()

Here we can examine our target rows before the update.
We can see that all these rows are missing data, we use this to get the index

In [16]:
data_rows_incomplete['stalk-root']

3984    ?
4023    ?
4076    ?
4100    ?
4104    ?
       ..
8119    ?
8120    ?
8121    ?
8122    ?
8123    ?
Name: stalk-root, Length: 2480, dtype: object

#### Create a series with the name of the columns and set the index of the rows
we will  copy the index of the incomplete rows so we can insert them back into the original data

In [17]:
series_update = pd.Series(missing_values, name='stalk-root', index=data_rows_incomplete.index)
series_update

3984    b
4023    b
4076    e
4100    b
4104    b
       ..
8119    e
8120    e
8121    e
8122    b
8123    e
Name: stalk-root, Length: 2480, dtype: object

In the main dataset we see there are some rows with data, and some rows which are still missing data with '?'

In [18]:
mushroom_data['stalk-root']

0       e
1       c
2       c
3       e
4       e
       ..
8119    ?
8120    ?
8121    ?
8122    ?
8123    ?
Name: stalk-root, Length: 8124, dtype: object

#### Update the imputed values back into the original dataset
this works becuase the series object we created includes the column namne and index for the rows

In [19]:
mushroom_data.update(series_update)

Finally, we can see that the rows which were missing data, have now been updated.

In [20]:
mushroom_data['stalk-root']

0       e
1       c
2       c
3       e
4       e
       ..
8119    e
8120    e
8121    e
8122    b
8123    e
Name: stalk-root, Length: 8124, dtype: object

We can verify that there are no longer any '?' values in our data

In [21]:
(mushroom_data['stalk-root']=='?').any()

False

# Concept Question 1
#### why dont we one-hot encode the respose data to train the KNN model:

since one-hot encoding adds a new dimension for each value, and the 'stalk-root' has six different valid values, the encoded result would have six different dimensions.

If our labels had multiple dimensions, our problem domain changes from a multi-class, to a multi-label problem;
which requires a more complex regression model and significanlty more computing resources to train, without providing any additional value to our solution.

In practice even using the LabelBinarizer seems to be incompatible with the KNeighborsClassifier which is returning a set of 4 distinct binary columns:

##### ValueError: y should be a 1d array, got an array of shape (2480, 4) instead.

---
### Create Test and Training Dataset

Starting with the updated and re-asembled dataset, we will create a new complete dataset of OneHot Features and LabelEncoded labels for training.

This time the label we are predicting is the 'edible' dimension, will be the 'edible' field.

In [22]:
mushroom_labels = mushroom_data['edible']
mushroom_features = mushroom_data.loc[:, ((mushroom_data.columns != 'edible')) ]

In [23]:
inferencing_labels_Encoded = encoder_Labels_inference.fit_transform(mushroom_labels)
inferencing_features_OneHot = encoder_OneHot_inference.fit_transform(mushroom_features)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    inferencing_features_OneHot, 
    inferencing_labels_Encoded, 
    test_size=0.33,
    random_state=42
)

---
### Random Forest with Full Dataset

In [25]:
random_forest = RandomForestClassifier(random_state=42)

In [26]:
%%time
random_forest.fit(X_train, y_train)

CPU times: user 377 ms, sys: 3.79 ms, total: 381 ms
Wall time: 382 ms


RandomForestClassifier(random_state=42)

In [27]:
predictions_random_forest = random_forest.predict(X_test)

accuracy_rfc = accuracy_score(y_test, predictions_random_forest)
precision_rfc = precision_score(y_test, predictions_random_forest)
recall_rfc = recall_score(y_test, predictions_random_forest)

print("Confusion Matrix: ")
print(confusion_matrix(y_test, predictions_random_forest))
print("Accuracy: " + str(accuracy_rfc))
print("Precision: " + str(precision_rfc))
print("Recall: " + str(recall_rfc))

Confusion Matrix: 
[[1378    0]
 [   0 1303]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0


---
### Logistic Regression with Full Dataset

In [28]:
logistic_regression = LogisticRegression() 

In [29]:
%%time
logistic_regression.fit(X_train, y_train)

CPU times: user 77.8 ms, sys: 1.8 ms, total: 79.6 ms
Wall time: 79 ms


LogisticRegression()

In [30]:
predictions_logistic_regression = logistic_regression.predict(X_test)

accuracy_lrg = accuracy_score(y_test, predictions_logistic_regression)
precision_lrg = precision_score(y_test, predictions_logistic_regression)
recall_lrg = recall_score(y_test, predictions_logistic_regression)

print("Confusion Matrix: ")
print(confusion_matrix(y_test, predictions_logistic_regression))
print("Accuracy: " + str(accuracy_lrg))
print("Precision: " + str(precision_lrg))
print("Recall: " + str(recall_lrg))

Confusion Matrix: 
[[1378    0]
 [   0 1303]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0


# Concept Question 2
#### Could we train these models by one-hot encoding the response data if we dropped the first parameter:


Yes, using the LabelBinarizer produces identical results in this case.

becuase there are only two potential values for 'edible', then if we one-hot encoded the data and dropped the first parameter, we would end up with a single binary dimension like 'poisonous' = True/False

However I was not able to actually make the OneHotEncoder object perform the fit.
The OneHotEncoder can return a sparse matrix which makes the RandomForest model throw an error when fitting:
###### "sparse multilabel-indicator for y is not supported"

The scikit-learn documentation for [OneHotEncoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) is clear about this:

"Note: a one-hot encoding of y labels should use a LabelBinarizer instead."

---
### Perform Principal Component Analysis

To reduce the number of features, we will fit a PCA model on entire featureset; 
Then use the trained PCA model to reduce the number dimensions (columns) in X_train and X_test

lets examine how many dimensions are in X_train; and we see that there are 116 dimensions to start with

In [31]:
X_train.shape

(5443, 116)

In [32]:
pca = PCA(n_components=0.95, random_state=42)
pca.fit(inferencing_features_OneHot.toarray())



PCA(n_components=0.95, random_state=42)

lets examine our PCA model components

In [33]:
pca.n_components_

41

We have reduced the number of encoded features from 116 down to 41.
So we have dropped 75 dimensions, which is a reduction of 64.65%

Now we will create reduced versions of our original training and test sets.

we do this so our next models will train on the exact same records (rows), but fewer features (columns)

In [34]:
X_train_reduced = pca.transform(X_train.toarray())
X_test_reduced = pca.transform(X_test.toarray())

In [35]:
X_train_reduced.shape

(5443, 41)

In [36]:
X_test_reduced.shape

(2681, 41)

---
### Random Forest Classifier on the Reduced features

In [37]:
random_forest_reduced = RandomForestClassifier(random_state=42)

In [38]:
%%time
random_forest_reduced.fit(X_train_reduced, y_train)

CPU times: user 1.71 s, sys: 2.94 ms, total: 1.71 s
Wall time: 1.72 s


RandomForestClassifier(random_state=42)

In [39]:
predictions_random_forest_reduced = random_forest_reduced.predict(X_test_reduced)

accuracy_rfc_reduced = accuracy_score(y_test, predictions_random_forest_reduced)
precision_rfc_reduced = precision_score(y_test, predictions_random_forest_reduced)
recall_rfc_reduced = recall_score(y_test, predictions_random_forest_reduced)

print("Confusion Matrix: ")
print(confusion_matrix(y_test, predictions_random_forest_reduced))
print("Accuracy: " + str(accuracy_rfc_reduced))
print("Precision: " + str(precision_rfc_reduced))
print("Recall: " + str(recall_rfc_reduced))

Confusion Matrix: 
[[1378    0]
 [   0 1303]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0


---
### Logistic Regression against the Reduced features

In [40]:
logistic_regression_reduced = LogisticRegression() 

In [41]:
%%time
logistic_regression_reduced.fit(X_train_reduced, y_train)

CPU times: user 67.6 ms, sys: 4.96 ms, total: 72.6 ms
Wall time: 37.1 ms


LogisticRegression()

In [42]:
predictions_logistic_regression_reduced = logistic_regression_reduced.predict(X_test_reduced)

accuracy_lrg_reduced = accuracy_score(y_test, predictions_logistic_regression_reduced)
precision_lrg_reduced = precision_score(y_test, predictions_logistic_regression_reduced)
recall_lrg_reduced = recall_score(y_test, predictions_logistic_regression_reduced)

print("Confusion Matrix: ")
print(confusion_matrix(y_test, predictions_logistic_regression_reduced))
print("Accuracy: " + str(accuracy_lrg_reduced))
print("Precision: " + str(precision_lrg_reduced))
print("Recall: " + str(recall_lrg_reduced))

Confusion Matrix: 
[[1372    6]
 [   8 1295]]
Accuracy: 0.9947780678851175
Precision: 0.9953881629515757
Recall: 0.9938603223330775


| Models | Item | Full Data | PCA Reduced |
| :----- | :--- | :-------- | :---------- |
| <b> Random Forest </b> | <b> Accuracy </b> | 1.0 | 1.0 |
| | <b> Precision </b>| 1.0 | 1.0 |
| | <b> Recall </b>| 1.0 | 1.0 |
| | <b> Time </b>| 381 ms | 1.71 s |
| <b> Logistic Regression </b> | <b> Accuracy </b> | 1.0 | 0.99477 |
| | <b> Precision </b>| 1.0 | 0.9953 |
| | <b> Recall </b>| 1.0 | 0.9938 |
| | <b> Time </b>| 79.6 ms | 72.6 ms |

# Discussion

I do not entirely understand why the Random Forest model took longer to train on a smaller dataset.

I do think it is interesting to see how the RandomForest and LogisticRegression models perform differently when we reduce the dataset.
While the RandomForest takes significantly longer to train with reduced data, it still performs with great accuracy.

The LogisticRegression model trains just a little faster, but begins to lose accuracy when we reduce the data.

It seems like the RandomForest model makes a worthwhile investment of computing resources to spend the additional time and likely producing a more optimal model. 